Data Ingestion stage

In [1]:

from collections import namedtuple
import os

In [2]:
%pwd

'/home/vikasrathod/vikasr/Courses_vikas_data/CV_Project/cnnClassifier/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vikasrathod/vikasr/Courses_vikas_data/CV_Project/cnnClassifier'

In [5]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [15]:
DataIngestionConfig

__main__.DataIngestionConfig

In [6]:
#config_entity.py
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [8]:
#configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
# import os
# import urllib.request as request
# from zipfile import ZipFile

In [26]:
#data_ingestion.py
import os
from urllib import request
from zipfile import ZipFile, BadZipFile, is_zipfile


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Check if the local zip file already exists
        if not os.path.exists(self.config.local_data_file):
            print(f"Downloading file from {self.config.source_URL}...")
            try:
                # Download the zip file from the GitHub raw URL
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                print(f"File downloaded to {self.config.local_data_file}")
            except Exception as e:
                print(f"Error downloading file: {e}")

    def _get_updated_list_of_files(self, list_of_files):
        # Filter files based on your criteria (e.g., .jpg files with "men" or "women" in the name)
        return [f for f in list_of_files if f.endswith(".jpg") and ("men" in f or "women" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        # Define the target file path
        target_filepath = os.path.join(working_dir, f)
        
        # Extract the file if it doesn't exist already
        if not os.path.exists(target_filepath):
            try:
                print(f"Extracting {f}...")
                zf.extract(f, working_dir)
            except Exception as e:
                print(f"Error extracting file {f}: {e}")
                return
        
        # Remove empty files
        if os.path.getsize(target_filepath) == 0:
            print(f"Removing empty file {target_filepath}...")
            os.remove(target_filepath)

    def unzip_and_clean(self):
        # Check if the local zip file exists
        if not os.path.exists(self.config.local_data_file):
            print(f"Local zip file not found at {self.config.local_data_file}")
            return
        
        # Check if the file is a valid zip file
        if not is_zipfile(self.config.local_data_file):
            print(f"Error: The file {self.config.local_data_file} is not a valid zip file.")
            return
        
        try:
            # Open the zip file
            with ZipFile(file=self.config.local_data_file, mode="r") as zf:
                list_of_files = zf.namelist()
                updated_list_of_files = self._get_updated_list_of_files(list_of_files)
                
                # Preprocess each file in the updated list
                for f in updated_list_of_files:
                    self._preprocess(zf, f, self.config.unzip_dir)  # type: ignore
                print("Unzipping and cleaning completed.")
        except BadZipFile:
            print(f"Error: The file {self.config.local_data_file} is not a valid zip file.")
        except Exception as e:
            print(f"Error during unzip and clean process: {e}")












#Training_pipeline.py
# Example usage
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Download the zip file
    data_ingestion.download_file()
    
    # Unzip and clean the files
    data_ingestion.unzip_and_clean()
except Exception as e:
    print(f"An error occurred: {e}")


[2024-12-19 16:11:31,708: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-19 16:11:31,710: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-19 16:11:31,711: INFO: common: created directory at: artifacts]
[2024-12-19 16:11:31,713: INFO: common: created directory at: artifacts/data_ingestion]
Extracting Classificationdata/test/men/136.jpg...
Extracting Classificationdata/test/women/119.jpg...
Extracting Classificationdata/train/men/0.jpg...
Extracting Classificationdata/train/men/2.jpg...
Extracting Classificationdata/train/men/3.jpg...
Extracting Classificationdata/train/women/10.jpg...
Extracting Classificationdata/train/women/6.jpg...
Extracting Classificationdata/train/women/9.jpg...
Unzipping and cleaning completed.
